In [ ]:
# !pip install jupyter-dash, dash_bootstrap_components colorlover

In [ ]:
import io
import base64

from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import dash_bootstrap_components as dbc


In [ ]:
layout = {
    'margin': {'b': 15, 't': 15},
    'xaxis': {'domain': [0, 0.9]},
    'yaxis': {'tickfont': {'color': '#636efa'}, 'title': {'font': {'color': '#636efa'}, 'text': 'Price'}},
    'yaxis2': {'anchor': 'x',
               'overlaying': 'y',
               'side': 'right',
               'tickfont': {'color': '#ef553b'},
               'title': {'font': {'color': '#ef553b'}, 'text': 'Days to cover'}},
    'yaxis3': {'anchor': 'free',
               'overlaying': 'y',
               'position': 0.95,
               'side': 'right',
               'tickfont': {'color': '#00cc96'},
               'title': {'font': {'color': '#00cc96'}, 'text': 'Short Interest'}},
}


In [ ]:
def get_timeseries(id):
    [ticker, exchange] = id.split('_')
    df = pd.read_csv(f'data/{ticker}-{exchange}.csv')
    df = df.drop([f'open {ticker} Price in USD', f'high {ticker} Price in USD',
                  f'low {ticker} Price in USD', 'Dividends', 'hidden'], axis=1)
    df = df.rename(columns={'Exchange Reported SI': 'SI',
                            f'close {ticker} Price in USD': 'Close', 'Days to cover 3m (on loan)': 'Days To Cover'})
    df = df.dropna(subset=['Close'])
    df['Date'] = pd.to_datetime(df['Date'])
    df.head
    df['Close 2 week rolling'] = df['Close'].rolling(10).mean()
    df = df.set_index('Date')
    first_SI_index = df['SI'].first_valid_index()
    df = df.loc[first_SI_index:]
    return df


def get_time_lag_plot(df):
    # time lagged cross correlation
    def crosscorr(datax, datay, lag=0, wrap=False):
        if wrap:
            shiftedy = datay.shift(lag)
            shiftedy.iloc[:lag] = datay.iloc[-lag:].values
            return datax.corr(shiftedy)
        else:
            return datax.corr(datay.shift(lag))

    d1 = df['Close']
    d2 = df['SI'].interpolate()
    rs = [crosscorr(d1, d2, lag) for lag in range(-60, 60)]
    # use absolute value as could be positve or negatively correlated
    offset = np.floor(len(rs)/2)-np.argmax(np.abs(rs))
    print(rs[np.argmax(np.abs(rs))])
    fig, ax = plt.subplots(figsize=(20, 5))
    ax.plot(rs)
    ax.axvline(np.ceil(len(rs)/2), color='k', linestyle='--', label='Center')


    ax.axvline(np.argmax(np.abs(rs)), color='r',
            linestyle='--', label='Peak synchrony')
    ax.set(title=f'Offset = {offset}', xlabel='Offset', ylabel='Pearson r')
    ax.set_xticks([0, 20, 40, 60, 80, 100, 120])
    ax.set_xticklabels([-60, -40, -20, 0, 20, 40, 60])
    plt.legend()
    buf = io.BytesIO()
    plt.savefig(buf, format = "png")
    plt.close()
    data = base64.b64encode(buf.getbuffer()).decode("utf8")
    return "data:image/png;base64,{}".format(data)



In [ ]:
stats_df = pd.read_csv('stats.csv')
stats_data = stats_df.to_dict('records')
for row in stats_data:
    row['id'] = row['ticker']

desc_df = stats_df.describe()
desc_df.reset_index(level=0, inplace=True)
desc_data = desc_df.to_dict('records')

# For DTC decile
dtc_decile = stats_df.groupby(pd.qcut(stats_df['Avg DTC'], 10))['Annualised return'].mean()
dtc_decile.index = dtc_decile.index.astype(str)

# For CTB
ctb_dropped = stats_df[stats_df['Avg CTB'] < 2]
ctb_decile = stats_df.groupby(pd.qcut(stats_df['Avg CTB'], 10))['Annualised return'].mean()
ctb_decile.index = ctb_decile.index.astype(str)

In [ ]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

server = app.server


def discrete_background_color_bins(df, n_bins=5, columns=['SI Pearson r', 'DTC Pearson r', 'CTB Pearson r']):
    import colorlover
    bounds = [i * (1.0 / n_bins) for i in range(n_bins + 1)]
    df_numeric_columns = df[columns]
    df_max = 1
    df_min = 0
    ranges = [
        ((df_max - df_min) * i) + df_min
        for i in bounds
    ]
    styles = []
    legend = []
    for i in range(1, len(bounds)):
        min_bound = ranges[i - 1]
        max_bound = ranges[i]
        backgroundColor = colorlover.scales[str(
            n_bins)]['seq']['Greens'][i - 1]
        color = 'white' if i > len(bounds) / 2. else 'inherit'

        for column in df_numeric_columns:
            styles.append({
                'if': {
                    'filter_query': (
                        '{{{column}}} >= {min_bound}' +
                        (' && {{{column}}} < {max_bound}' if (
                            i < len(bounds) - 1) else '')
                    ).format(column=column, min_bound=min_bound, max_bound=max_bound),
                    'column_id': column
                },
                'backgroundColor': backgroundColor,
                'color': color
            })
            styles.append({
                'if': {
                    'filter_query': (
                        '{{{column}}} <= {min_bound}' +
                        (' && {{{column}}} > {max_bound}' if (
                            i < len(bounds) - 1) else '')
                    ).format(column=column, min_bound=-min_bound, max_bound=-max_bound),
                    'column_id': column
                },
                'backgroundColor': backgroundColor,
                'color': color
            })
        legend.append(
            html.Div(style={'display': 'inline-block', 'width': '60px'}, children=[
                html.Div(
                    style={
                        'backgroundColor': backgroundColor,
                        'borderLeft': '1px rgb(50, 50, 50) solid',
                        'height': '10px'
                    }
                ),
                html.Small(round(min_bound, 2), style={'paddingLeft': '2px'})
            ])
        )

    return (styles, html.Div(legend, style={'padding': '5px 0 5px 0'}))


(styles, legend) = discrete_background_color_bins(stats_df)


def corr_histo():
    fig = make_subplots(rows=3, cols=1)
    fig.add_trace(
        go.Histogram(x=stats_df["SI Pearson r"], name="SI Pearson r"),
        row=1, col=1,
    )
    fig.add_trace(
        go.Histogram(x=stats_df["DTC Pearson r"], name="DTC Pearson r"),
        row=2, col=1
    )
    fig.add_trace(
        go.Histogram(x=stats_df["CTB Pearson r"], name="CTB Pearson r"),
        row=3, col=1,
    )
    fig.update_layout(height=800, width=600, title_text="Distribution plots")
    return fig


def offset_histo():
    fig = make_subplots(rows=3, cols=1)
    fig.add_trace(
        go.Histogram(x=stats_df["SI offset"], name="SI offset"),
        row=1, col=1,
    )
    fig.add_trace(
        go.Histogram(x=stats_df["DTC offset"], name="DTC offset"),
        row=2, col=1
    )
    fig.add_trace(
        go.Histogram(x=stats_df["CTB offset"], name="CTB offset"),
        row=3, col=1,
    )

    fig.update_layout(height=800, width=600,
                      title_text="Distribution plots for offset")
    return fig


app.layout = dbc.Container(style={'paddingTop': 30}, children=[
    dcc.Graph(figure=px.histogram(
        stats_df, x="Avg DTC", title="Average Days to Cover")),
    dcc.Graph(figure=px.scatter(stats_df, x='Avg DTC',
              y='Annualised return', title="DTC against returns scatter")),
    dcc.Graph(figure=px.bar(dtc_decile, y='Annualised return', labels={
        "Annualised return": "Avg annualised return",
    }, title="Average return by DTC decile")),
    dcc.Graph(figure=px.histogram(
        ctb_dropped, x="Avg CTB", title="Average Cost to Borow")),
    dcc.Graph(figure=px.scatter(ctb_dropped, x='Avg CTB',
              y='Annualised return', title="CTB against returns scatter")),
    dcc.Graph(figure=px.bar(ctb_decile, y='Annualised return',  labels={
        "Annualised return": "Avg annualised return",
    }, title="Average return by CTB decile")),
    dcc.Graph(figure=corr_histo()),
    dcc.Graph(figure=offset_histo()),
    html.H3(
        children="Overall stats"
    ),
    dash_table.DataTable(
        id='describe',
        columns=[{"name": i, "id": i} for i in desc_df.columns],
        data=desc_data,
        style_table={'overflowX': 'auto', 'minWidth': '100%'},
        fixed_columns={'headers': True, 'data': 1},
    ),
    html.H3(
        children="List of stocks"
    ),
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in stats_df.columns],
        data=stats_data,
        filter_action="native",
        sort_action="native",
        page_action="native",
        page_current=0,
        page_size=10,
        style_data_conditional=styles,
        style_table={'overflowX': 'auto', 'minWidth': '100%'},
        fixed_columns={'headers': True, 'data': 1},
    ),
    html.P(id='selected_ticker'),
    html.Div([
        dcc.Graph(
            id='ts_graph',
        )
    ]),
    html.Ul(id='list_correl', children=[
            html.Li(id="dtc_correl"), html.Li(id="si_correl")]),
    html.Img(id='time_lag_plot')
])

fig_graph = go.Figure()
fig_graph.update_layout(**layout)


@app.callback(Output('selected_ticker', 'children'), Output('ts_graph', 'figure'), Output('time_lag_plot', 'src'), Input('table', 'active_cell'))
def update_graphs(active_cell):
    if active_cell:
        ts = get_timeseries(active_cell['row_id'])

        fig_graph.data = []

        fig_graph.add_trace(go.Scatter(
            x=ts.index,
            y=ts['Close'],
            name="Close"
        ))

        fig_graph.add_trace(go.Scatter(
            x=ts.index,
            y=ts['Days To Cover'],
            name="Days To Cover",
            yaxis='y2',
        ))

        fig_graph.add_trace(go.Scatter(
            x=ts.index,
            y=ts['SI'],
            name="Short Interest",
            yaxis='y3',
            mode='lines+markers',
            connectgaps=True
        ))

        img = get_time_lag_plot(ts)

        return "Selected ticker: " + str(active_cell['row_id'].split('_')), fig_graph, img
    else:
        return "Select a row", fig_graph, ''


@app.callback(Output('dtc_correl', 'children'), Output('si_correl', 'children'), Input('table', 'active_cell'))
def update_list(active_cell):
    if active_cell:
        ticker = active_cell['row_id']
        si_r, si_p = stats_df[stats_df['ticker'] ==
                              ticker]['SI Pearson r'].values[0], stats_df[stats_df['ticker'] == ticker]['SI p-value (r)'].values[0]
        dtc_r, dtc_p = stats_df[stats_df['ticker'] == ticker]['DTC Pearson r'].values[
            0], stats_df[stats_df['ticker'] == ticker]['DTC p-value (r)'].values[0]
        return f"SI Pearson R: {si_r}, p-value: {si_p}", f"DTC Pearson R: {dtc_r}, p-value: {dtc_p}"
    else:
        return '', ''


In [ ]:
app.run_server(mode='inline')  # use mode='external' to run on localhost:8050
